# Bump test ComCam campaign

Craig Lage  04-Apr-25

In [ ]:
import asyncio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst_efd_client import EfdClient
from lsst.ts.xml.enums.MTM1M3 import BumpTest
import logging
logging.basicConfig(level='INFO')
mlogger = logging.getLogger('matplotlib')
mlogger.setLevel(logging.WARNING)

# These are the subroutines that query the EFD and do the work

In [ ]:
def max_error(errors):
    return np.max([np.max(errors), np.max(errors * -1.0)])
    
def calc_rms_times(t_start):
    # This is the date of the IRQ change, which changed the bump test timings
    change_date = Time("2024-10-12T00:00", format='isot', scale='utc')
    if t_start < change_date:
        rms_t1 = 3.0
        rms_t2 = 4.0
        rms_t3 = 10.0
        rms_t4 = 11.0
    else:
        rms_t1 = 2.9
        rms_t2 = 3.9
        rms_t3 = 9.3
        rms_t4 = 10.3
    return [rms_t1, rms_t2, rms_t3, rms_t4]
    
def rms_error(times, errors, rms_times):
    [rms_t1, rms_t2, rms_t3, rms_t4] = rms_times

    error = 0.0
    num = 0
    for i, t in enumerate(times):
        if (t > rms_t1 and t < rms_t2) or (t > rms_t3 and t < rms_t4):
            num += 1
            error += errors[i]**2
    if num == 0:
        return np.nan
    else:
        return np.sqrt(error / num)
        
async def calc_bumps_and_errors(bump, bt_result, force, follow, applied, p_s, axs=None):
    # If a set of axes is supplied, it will build the detailed plots
    BUMP_TEST_DURATION = 14.0  # seconds
    max_x_ticks = 25
    measured_forces_times = []
    measured_forces_values = []
    following_error_values = []
    applied_forces_times = []
    applied_forces_values = []
    t_starts = []
    qTimes = []

    results = bt_result[bt_result[bump] == BumpTest.TESTINGPOSITIVE]
    for bt_index in range(len(results)):
        t_start = Time(
            bt_result[bt_result[bump] == BumpTest.TESTINGPOSITIVE][
                "timestamp"
            ].values[bt_index]
            - 1.0,
            format="unix_tai",
            scale="tai",
        )
        t_starts.append(t_start.isot.split('.')[0])
        t_end = Time(
            t_start + TimeDelta(BUMP_TEST_DURATION, format="sec"),
            format="unix_tai",
            scale="tai",
        )
        if axs is None:
            t0 = Time.now()
            measured_forces = await client.select_time_series(\
                        "lsst.sal.MTM1M3.forceActuatorData", \
                        [follow, "timestamp"], t_start.utc, t_end.utc)
            t1 = Time.now()
            deltaT = (t1 - t0).value * 86400 * 1000
            qTimes.append(deltaT)

        else:
            measured_forces = await client.select_time_series(\
                        "lsst.sal.MTM1M3.forceActuatorData", \
                        [force, follow, "timestamp"], t_start.utc, t_end.utc)
            applied_forces = await client.select_time_series(\
                "lsst.sal.MTM1M3.appliedForces", \
                [applied, "timestamp"], t_start.utc, t_end.utc)
    
            t0 = measured_forces["timestamp"].values[0]
            applied_forces["timestamp"] -= t0
            measured_forces_value = measured_forces[force].values
            if p_s != "Primary":
                if 'MINUS' in p_s:
                    measured_forces_value = np.array(measured_forces_value) / -np.sqrt(2.0)
                if 'PLUS' in p_s:
                    measured_forces_value = np.array(measured_forces_value) / np.sqrt(2.0)
            measured_forces_values.append(measured_forces_value)
            applied_forces_time = applied_forces["timestamp"].values
            applied_forces_times.append(applied_forces_time)
            applied_forces_value = applied_forces[applied].values
            applied_forces_values.append(applied_forces_value)
        if len(measured_forces) == 0:
            continue
        t0 = measured_forces["timestamp"].values[0]
        measured_forces["timestamp"] -= t0
        measured_forces_time = measured_forces["timestamp"].values
        measured_forces_times.append(measured_forces_time)
        following_error_value = measured_forces[follow].values
        following_error_values.append(following_error_value)
    times = []
    max_errors = []
    rms_errors = []
    for i in range(len(measured_forces_times)):
        times.append(t_starts[i])
        max_errors.append(max_error(following_error_values[i]))
        rms_errors.append(rms_error(measured_forces_times[i], \
                                    following_error_values[i], calc_rms_times(t_starts[i])))
    if axs is None:
        return [times, rms_errors, max_errors, qTimes]

    if p_s == "Primary":
        plot_index = 0
    else:
        plot_index = 1
    
    axs[0][plot_index].set_title(f"Actuator {id} {p_s} forces vs time")
    axs[0][plot_index].plot(applied_forces_times[0], applied_forces_values[0])
    for i in range(len(measured_forces_times)):
        axs[0][plot_index].plot(measured_forces_times[i], measured_forces_values[i], label=t_starts[i])
    axs[0][plot_index].set_xlim(0,14.0)
    axs[0][plot_index].set_xlabel("Time(sec.)")
    axs[0][plot_index].set_ylim(-400, 400)
    axs[0][plot_index].set_ylabel("Force(N)")
    this_index = -1
    axs[1][plot_index].set_title(f"Actuator {id} {p_s} forces {t_starts[this_index]}")
    axs[1][plot_index].plot(applied_forces_times[this_index], applied_forces_values[this_index])
    axs[1][plot_index].plot(measured_forces_times[this_index], measured_forces_values[this_index], label="Forces")
    axs[1][plot_index].plot(measured_forces_times[this_index], following_error_values[this_index], label="Following error")
    axs[1][plot_index].legend()
    axs[1][plot_index].set_xlim(0,14.0)
    axs[1][plot_index].set_xlabel("Time(sec.)")
    axs[1][plot_index].set_ylim(-400, 400)
    axs[1][plot_index].set_ylabel("Force(N)")
    [rms_t1, rms_t2, rms_t3, rms_t4] = calc_rms_times(t_starts[this_index])
    axs[1][plot_index].axvline(rms_t1, ls='--', color='black')
    axs[1][plot_index].axvline(rms_t2, ls='--', color='black')
    axs[1][plot_index].axvline(rms_t3, ls='--', color='black')
    axs[1][plot_index].axvline(rms_t4, ls='--', color='black')

    axs[2][plot_index].set_title(f"Actuator {id} {p_s} following errors")
    times = []
    max_errors = []
    rms_errors = []
    for i in range(len(measured_forces_times)):
        times.append(t_starts[i])
        max_errors.append(max_error(following_error_values[i]))
        rms_errors.append(rms_error(measured_forces_times[i], \
                                    following_error_values[i], calc_rms_times(t_starts[i])))
    # Cut down the number of times ticks to keep it readable
    # But make sure you keep the most recent one.
    n_subset = int(len(times) / max_x_ticks) + 1
    x_ticks = times[::n_subset]
    if times[-1] not in x_ticks:
        x_ticks.append(times[-1])
        x_ticks.remove(x_ticks[-2])
    axs[2][plot_index].plot(times, rms_errors, marker='x', color='blue', label="RMS")
    axs[2][plot_index].plot(times, max_errors, marker='+', color='green', label="Max")
    axs[2][plot_index].set_ylim(0,1000)
    axs[2][plot_index].set_yscale('symlog', linthresh=10)
    axs[2][plot_index].set_yticks([0,2,4,6,8,10,50,100,500,1000])
    axs[2][plot_index].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axs[2][plot_index].set_xticks(x_ticks)
    axs[2][plot_index].tick_params(axis='x', rotation=90)
    axs[2][plot_index].set_ylabel("RMS and Max errors (N)")
    axs[2][plot_index].legend()
    return [times, rms_errors, max_errors]

async def calc_actuator_error(client, fa_id, bt_results, fig=None):
    # If a figure is supplied, it will build the plots
    if fig:
        axs = fig.subplots(3,2)
        plt.gcf().subplots_adjust(bottom=0.25, wspace=0.3, hspace=0.3)
    else:
        axs = None
    
    # Grab the Force Actuator Data from its ID
    fa_data = force_actuator_from_id(fa_id)
    bt_result = bt_results[bt_results["actuatorId"] == fa_id]
    # First the primary forces
    bump = f"primaryTest{fa_data.index}"
    force = f"primaryCylinderForce{fa_data.index}"
    applied = f"zForces{fa_data.z_index}"
    follow = f"primaryCylinderFollowingError{fa_data.index}"
    [ptimes, prms_errors, pmax_errors, pqTimes] = \
        await calc_bumps_and_errors(bump, bt_result, force, follow, applied, "Primary", axs)

    # Now the secondary  forces  
    if fa_data.actuator_type.name == "DAA":
        bump = f"secondaryTest{fa_data.s_index}"
        force = f"secondaryCylinderForce{fa_data.s_index}"
        follow = f"secondaryCylinderFollowingError{fa_data.s_index}"
        secondary_name = fa_data.orientation.name
        if secondary_name in ["X_PLUS", "X_MINUS"]:
            applied = f"xForces{fa_data.x_index}"
        elif secondary_name in ["Y_PLUS", "Y_MINUS"]:
            applied = f"yForces{fa_data.y_index}"
        else:
            raise ValueError(f"Unknown secondary name {secondary_name}")
            
        [stimes, srms_errors, smax_errors, sqTimes] = \
            await calc_bumps_and_errors(bump, bt_result, force, follow, applied, secondary_name, axs)
    else:
        stimes = []; srms_errors = []; smax_errors = []; sqTimes = []
    if not fig:
        return [ptimes, prms_errors, pmax_errors, stimes, srms_errors, smax_errors, pqTimes, sqTimes]
    else:
        return


# Now we query the EFD to get the times of all of the bump tests for the past days_to_plot days.

In [ ]:
start = Time("2024-10-24T00:00:00", scale='utc')
end = Time("2024-12-25T00:00:00", scale='utc')
client = EfdClient('usdf_efd')
bumps = await client.select_time_series(\
                    "lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", \
                    ['*'], start, end)
ids = []
for index in range(len(FATable)):
    id = FATable[index].actuator_id
    ids.append(id)


# Now we build the trend of recent failures

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,2, figsize=(10,10))
plt.suptitle("Actuator Recent History\nNot plotted -> All green", fontsize=18)
plot_list = [[0,0,"Primary RMS Errors", 0, 1, 7, 14],
             [1,0,"Primary Max Errors", 0, 2, 250, 300],
             [0,1,"Secondary RMS Errors", 3, 4, 7, 14],
             [1,1,"Secondary Max Errors", 3, 5, 315, 420]]
ns = np.zeros(4) # Number of bad ids per plot
bad_ids = [[],[],[],[]] # List of bad_ids per plot
bad_ids_last_point = []
min_times = np.ones(4) * 1.0E12 # Min/Max times in unix_tai
max_times = np.zeros(4) # Min/Max times in unix_tai
min_time_list = [[],[],[],[]] # Min/Max times in isot
max_time_list = [[],[],[],[]] # Min/Max times in isot
qTimes = []
qDates = []
for id in ids:
    data = await calc_actuator_error(client, id, bumps)
    qTimes += data[6]
    qDates += data[0]
    qTimes += data[7]
    qDates += data[3]
    for m, [xplot, yplot, title, time_index, error_index, yellow_limit, red_limit] in enumerate(plot_list):
        errors = data[error_index]
        if len(errors) > 0:
            max = np.max(errors)
        else:
            max = 0.0
        if max > yellow_limit:
            bad_ids[m].append(id)
        else:
            continue

        times = data[time_index]
        
        actual_times_red = []
        actual_times_yellow = []
        actual_times_green = []
        yaxis_red = []
        yaxis_yellow = []
        yaxis_green = []
        n_times = len(times)    
        for i, time in enumerate(times):
            act_time = Time(time, format='isot').unix_tai
            if act_time < min_times[m]:
                min_times[m] = act_time
                min_time_list[m] = time
            if act_time > max_times[m]:
                max_times[m] = act_time
                max_time_list[m] = time
            
            if errors[i] > red_limit:
                actual_times_red.append(act_time)
                yaxis_red.append(ns[m])
                if i == n_times - 1:
                    bad_ids_last_point.append(id)
            elif errors[i] > yellow_limit:
                actual_times_yellow.append(act_time)
                yaxis_yellow.append(ns[m])
                if i == n_times - 1:
                    bad_ids_last_point.append(id)
            else:
                actual_times_green.append(act_time)
                yaxis_green.append(ns[m])

        axs[xplot][yplot].set_title(title)
        axs[xplot][yplot].scatter(actual_times_green, yaxis_green, color='green')
        axs[xplot][yplot].scatter(actual_times_yellow, yaxis_yellow, color='gold')
        axs[xplot][yplot].scatter(actual_times_red, yaxis_red, color='red')
        ns[m] += 1
for m, [xplot, yplot, title, time_index, error_index, yellow_limit, red_limit] in enumerate(plot_list):
    axs[xplot][yplot].set_yticks(list(range(len(bad_ids[m]))), bad_ids[m])
    axs[xplot][yplot].set_xticks([min_times[m], max_times[m]],[min_time_list[m], \
                                                               max_time_list[m]], rotation=10)        
bad_ids_last_point = list(set(bad_ids_last_point)) # Remove duplicates
#plt.show()
plt.savefig("/home/c/cslage/u/MTM1M3/data/Bump_Test_ComCam_04Apr25.png")